<a href="https://colab.research.google.com/github/orifelszer/CrimeData/blob/eden-branch/Supervised_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crimes Data in Israel (2019-2024)

#### by: Eden Shmuel, Oriana Feltzer

***

# Importing necessary libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import glob

In [7]:
import requests
import zipfile
import pandas as pd

def download_and_read_csv_from_zip(url, csv_filename):
    # הורדת הקובץ מהאינטרנט
    response = requests.get(url)
    zip_path = "temp.zip"  # שמירת הקובץ באופן זמני
    with open(zip_path, "wb") as file:
        file.write(response.content)

    # פתיחת ה-ZIP וקריאת הקובץ
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        with zip_ref.open(csv_filename) as file:
            return pd.read_csv(file)

# קריאת הקבצים מהמאגר (הורדה זמנית)
X_train = download_and_read_csv_from_zip(
    'https://github.com/orifelszer/CrimeData/raw/main/X_train.zip', 'X_train.csv')

X_test = download_and_read_csv_from_zip(
    'https://github.com/orifelszer/CrimeData/raw/main/X_test.zip', 'X_test.csv')

y_train = download_and_read_csv_from_zip(
    'https://github.com/orifelszer/CrimeData/raw/main/y_train.zip', 'y_train.csv').values.ravel()

y_test = download_and_read_csv_from_zip(
    'https://github.com/orifelszer/CrimeData/raw/main/y_test.zip', 'y_test.csv').values.ravel()

print("הנתונים נטענו בהצלחה!")



הנתונים נטענו בהצלחה!


In [8]:
# ייבוא הספריות הדרושות
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from itertools import product
from sklearn.ensemble import RandomForestClassifier

In [10]:
# בחירת התכונות על סמך הדגימה בלבד
selector = SelectKBest(score_func=chi2, k=10)
selector.fit(X_train, y_train)

# החלת הבחירה על כל סט האימון והבדיקה
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

In [11]:
# 📈 מודל בסיסי ללא שיפורים (Baseline)
sgd_model = SGDClassifier(loss='log_loss', class_weight='balanced', max_iter=100, random_state=42)
sgd_model.fit(X_train_selected, y_train)

# 📊 חיזוי על סט הבדיקה
y_pred_baseline = sgd_model.predict(X_test_selected)

# ✅ הערכת ביצועי המודל
accuracy_baseline = accuracy_score(y_test, y_pred_baseline)
print(f"Baseline Test Accuracy: {accuracy_baseline:.4f}")

Baseline Test Accuracy: 0.2308


In [12]:
# 🔁 קרוס-ולידציה על המודל הבסיסי
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
cv_scores = cross_val_score(sgd_model, X_train_selected, y_train, cv=cv, scoring='accuracy')

# תוצאות
print(f"Cross-Validation Scores: {cv_scores}")
print(f"Average Accuracy: {np.mean(cv_scores):.4f}")

Cross-Validation Scores: [0.20208582 0.26519561 0.18328286]
Average Accuracy: 0.2169


In [ ]:
# 📊 פונקציה למציאת הפרמטרים האופטימליים של SGDClassifier
def find_best_sgd_combination(X_train, y_train, X_test, y_test, alphas, max_iters, tols, k_features, cv_folds=3):
    best_score = 0
    best_params = {}
    results = []

    # לולאה על כל הקומבינציות האפשריות
    for alpha, max_iter, tol, k in product(alphas, max_iters, tols, k_features):

        # 🧩 בחירת התכונות
        selector = SelectKBest(score_func=chi2, k=k)
        selector.fit(X_train, y_train)
        X_train_selected = selector.transform(X_train)
        X_test_selected = selector.transform(X_test)

        # 📈 בניית מודל SGD עם הפרמטרים שנבדקים
        model = SGDClassifier(
            loss='log_loss',
            alpha=alpha,
            max_iter=max_iter,
            tol=tol,
            random_state=42,
            class_weight='balanced'
        )

        # 🔁 קרוס-ולידציה
        cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
        cv_scores = cross_val_score(model, X_train_selected, y_train, cv=cv, scoring='f1_macro')

        # אימון המודל על כל הדאטה
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)

        # 📊 חישוב מדדים
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='macro')
        recall = recall_score(y_test, y_pred, average='macro')
        f1 = f1_score(y_test, y_pred, average='macro')

        # שמירת התוצאות בכל הרצה
        results.append({
            'alpha': alpha,
            'max_iter': max_iter,
            'tol': tol,
            'features': k,
            'cv_f1_score': np.mean(cv_scores),
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1
        })

        # בדיקה אם התוצאה הנוכחית היא הטובה ביותר
        if f1 > best_score:
            best_score = f1
            best_params = {
                'alpha': alpha,
                'max_iter': max_iter,
                'tol': tol,
                'features': k,
                'f1': f1
            }

    #הדפסת התוצאה הטובה ביותר
    print(f"\n📊 Best Parameters Found:")
    print(f"Alpha: {best_params['alpha']}, Max Iter: {best_params['max_iter']}, Tol: {best_params['tol']}, Features: {best_params['features']}")
    print(f"Best F1-Score: {best_params['f1']:.4f}")

    #החזרת כל התוצאות והפרמטרים הטובים ביותר
    return results, best_params

In [ ]:
# פרמטרים ממוקדים יותר כדי למנוע קריסות
alphas = [0.001]
max_iters = [100]
tols = [1e-3]
k_features = [10]

# קריאה לפונקציה עם הפרמטרים המצומצמים
results, best_params = find_best_sgd_combination(X_train, y_train, X_test, y_test, alphas, max_iters, tols, k_features)

# הדפסת הפרמטרים הטובים ביותר שנמצאו
print("Best Parameters:", best_params)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [ ]:
# def find_best_random_forest_combination(X_train, y_train, X_test, y_test, n_estimators_list, max_depth_list, k_features, cv_folds=3):
#     best_score = 0
#     best_params = {}
#     results = []

#     # לולאה על כל הקומבינציות האפשריות
#     for n_estimators, max_depth, k in product(n_estimators_list, max_depth_list, k_features):

#         # 🧩 בחירת התכונות
#         selector = SelectKBest(score_func=chi2, k=k)
#         selector.fit(X_train, y_train)
#         X_train_selected = selector.transform(X_train)
#         X_test_selected = selector.transform(X_test)

#         # 📈 בניית מודל Random Forest עם הפרמטרים שנבדקים
#         model = RandomForestClassifier(
#             n_estimators=n_estimators,
#             max_depth=max_depth,
#             random_state=42,
#             class_weight='balanced')

#         # 🔁 קרוס-ולידציה
#         cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
#         cv_scores = cross_val_score(model, X_train_selected, y_train, cv=cv, scoring='f1_macro')

#         # אימון המודל על כל הדאטה
#         model.fit(X_train_selected, y_train)
#         y_pred = model.predict(X_test_selected)

#         # 📊 חישוב מדדים
#         accuracy = accuracy_score(y_test, y_pred)
#         precision = precision_score(y_test, y_pred, average='macro')
#         recall = recall_score(y_test, y_pred, average='macro')
#         f1 = f1_score(y_test, y_pred, average='macro')

#         # שמירת התוצאות בכל הרצה
#         results.append({
#             'n_estimators': n_estimators,
#             'max_depth': max_depth,
#             'features': k,
#             'cv_f1_score': np.mean(cv_scores),
#             'accuracy': accuracy,
#             'precision': precision,
#             'recall': recall,
#             'f1': f1})

#         # בדיקה אם התוצאה הנוכחית היא הטובה ביותר
#         if f1 > best_score:
#             best_score = f1
#             best_params = {
#                 'n_estimators': n_estimators,
#                 'max_depth': max_depth,
#                 'features': k,
#                 'f1': f1}

#     # ✅ הדפסת התוצאה הטובה ביותר
#     print(f"\n📊 Best Parameters Found:")
#     print(f"N Estimators: {best_params['n_estimators']}, Max Depth: {best_params['max_depth']}, Features: {best_params['features']}")
#     print(f"Best F1-Score: {best_params['f1']:.4f}")

#     # ✅ החזרת כל התוצאות והפרמטרים הטובים ביותר
#     return results, best_params

In [ ]:
# # דוגמה לפרמטרים אפשריים לבדיקה
# n_estimators_list = [50, 100, 200]
# max_depth_list = [10, 15, 20]  # None משמע ללא הגבלת עומק
# k_features = [10, 20, 30]

# # קריאה לפונקציה עם כל הקומבינציות
# results, best_params = find_best_random_forest_combination(X_train, y_train, X_test, y_test, n_estimators_list, max_depth_list, k_features)

# # הדפסת הקומבינציה הטובה ביותר
# print("Best Parameters:", best_params)

*****